In [13]:
import os
import json
import pandas as pd

from datetime import datetime
from utils.config import DRLConfig
from utils.drl_train import training_pipeline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_ret = pd.read_parquet("../data/returns.parquet")
df_prices = pd.read_parquet("../data/prices.parquet")
df_vol = pd.read_parquet("../data/vola.parquet")

To view the logs:
1. Open a terminal or command prompt.
2. Navigate to the directory *containing* the `logs` directory (i.e., the root of this repository).
3. Run the command: `tensorboard --logdir logs/`
4. Open the URL provided by TensorBoard (usually http://localhost:6006/) in your web browser.

You should see experiments named like `PPO_WindowX_AgentY_SeedZ`.

In [3]:
# Create timestamp for this run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create configuration
config = DRLConfig(
    # Window configuration
    n_windows=5,  # 10 in paper
    agents_per_window=5,  # 5 in paper
    base_start_year=2006,
    # Environment parameters
    env_window_size=60,
    transaction_cost=0.0,
    initial_balance=100_000,
    reward_scaling=1.0,
    eta_dsr=1 / 252,
    # Training parameters
    n_envs=10,
    total_timesteps_per_round=7_500_000,  # 7_500_000 in paper
    n_steps_per_env=252 * 3,
    batch_size=1260,
    n_epochs=16,
    gamma=0.9,
    gae_lambda=0.9,
    clip_range=0.25,
    log_std_init=-1.0,
    # Learning rate parameters
    initial_lr=3e-4,
    final_lr=1e-5,
    # Paths
    model_save_dir=f"../models/{timestamp}",
    tensorboard_log_dir=f"../logs/{timestamp}",
)

In [15]:
# Save config as JSON
from dataclasses import asdict
config_dict = asdict(config)
config_json_path = os.path.join(config.model_save_dir, f"config_{timestamp}.json")
os.makedirs(config.model_save_dir, exist_ok=True)
with open(config_json_path, 'w') as f:
    json.dump(config_dict, f, indent=4)
print(f"\nConfiguration saved to: {config_json_path}")


Configuration saved to: ../models/20250603_011024/config_20250603_011024.json


In [ ]:
# Run training pipeline
results, backtest_portfolio = training_pipeline(
    drl_config=config, df_prices=df_prices, df_ret=df_ret, df_vol=df_vol
)

In [12]:
for idx, p in backtest_portfolio.items():
    p.get_history().to_csv(f"{idx}_portfolio.csv", index=False)

In [6]:
results_filename = f"backtest_results_summary_{timestamp}.csv"
results_save_path = os.path.join(config.model_save_dir, results_filename)

results_df = pd.DataFrame(results)
results_df.to_csv(results_save_path, index=False)
print(f"\nBacktest results summary saved to: {results_save_path}")
print("\nFinal Results DataFrame:")
results_df.head()


Backtest results summary saved to: ../models/20250603_011024/backtest_results_summary_20250603_011024.csv

Final Results DataFrame:


,window,best_agent_path,n_eval_episodes,mean_reward,std_reward,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,Kurtosis,Tail ratio,Daily value at risk (95%),Portfolio turnover (in %),final_portfolio_value_first_episode
0,1,agent_seed0_valrew-8,1,-23.997776,0.0,0.076154,0.074901,0.100707,0.779247,1.011211,0.908507,-0.075310,1.139255,1.188347,-0.064170,0.705649,1.118682,-0.010244,42.383040,107490
1,2,agent_seed7_valrew-24,1,3.697691,0.0,0.217289,0.215391,0.090812,2.211520,4.238095,0.916748,-0.051270,1.428264,3.042375,-0.626832,1.273549,0.935650,-0.009891,41.387286,121539
2,3,agent_seed13_valrew6,1,21.616601,0.0,0.111844,0.110909,0.090601,1.215784,1.594303,0.916925,-0.070152,1.231374,1.582137,-0.427063,1.627926,0.791557,-0.010075,44.305055,111091
3,4,agent_seed15_valrew22,1,10.463516,0.0,-0.007043,-0.006987,0.124475,0.005489,-0.070600,0.889304,-0.099761,1.000921,0.008274,-0.083624,1.107017,0.908071,-0.013413,46.188586,99301
4,5,agent_seed24_valrew10,1,49.334480,0.0,0.198433,0.196712,0.105931,1.762439,3.527837,0.904216,-0.056248,1.361572,2.415603,-0.374161,2.311202,1.245118,-0.009740,48.483642,119671
